## Import Libraries

In [1]:
import os
import webbrowser
from neo4j import GraphDatabase
from pyvis.network import Network
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import time

## Load Data

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/movie-config.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Cypher Code

In [5]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
cypher = """
MATCH (p:Person {name: $name})-[r:ACTED_IN]->(m:Movie)
RETURN m.title AS title, r.role AS role
"""
name = "Tom Hanks"

records, summary, keys = driver.execute_query( # (1)
    cypher,    # (2)
    name=name  # (3)
)

In [7]:
print(keys)

['title', 'role']


In [8]:
print("Query Summary:")
print("Query Type:", summary.query_type)
print("Query Text:", summary.query)

print("\nCounters:")
for key, value in summary.counters.__dict__.items():
    if value:  # Only show non-zero counters
        print(f"{key}: {value}")

print("\nQuery Plan:")
if summary.plan:
    print(summary.plan)  # or use summary.plan.operator_type for plan root operator
else:
    print("No plan available.")

print("\nProfiled Plan:")
if summary.profile:
    print(summary.profile)  # or access attributes like profile.operator_type
else:
    print("No profile available.")

Query Summary:
Query Type: r
Query Text: 
MATCH (p:Person {name: $name})-[r:ACTED_IN]->(m:Movie)
RETURN m.title AS title, r.role AS role


Counters:

Query Plan:
No plan available.

Profiled Plan:
No profile available.


In [9]:
# RETURN m.title AS title, r.role AS role
for record in records:
    print("*" * 40)
    print(f"Title: {record['title']}")
    print(f"Role: {record['role']}")
    print("*" * 40)

****************************************
Title: Punchline
Role: Steven Gold
****************************************
****************************************
Title: Catch Me If You Can
Role: Carl Hanratty
****************************************
****************************************
Title: Dragnet
Role: Pep Streebeck
****************************************
****************************************
Title: Saving Mr. Banks
Role: Walt Disney
****************************************
****************************************
Title: Bachelor Party
Role: Rick Gassko
****************************************
****************************************
Title: Volunteers
Role: Lawrence Whatley Bourne III
****************************************
****************************************
Title: Man with One Red Shoe, The
Role: Richard Harlan Drew
****************************************
****************************************
Title: Splash
Role: Allen Bauer
****************************************
*

In [10]:
cypher = """
MATCH (m:Movie {title: $title})<-[:ACTED_IN]-(p)
RETURN p.name AS actor
"""
params = {"title": "Toy Story"}

res, summary, keys = driver.execute_query(

    cypher,
    **params
)

In [11]:
for record in res:
    print(record["actor"])

Jim Varney
Tim Allen
Tom Hanks
Don Rickles


In [12]:
temp_movie = "Inception"

In [13]:
records, summary, keys = driver.execute_query("""
MATCH path = (person:Person)-[actedIn:ACTED_IN]->(movie:Movie {title: $title})
RETURN path, person, actedIn, movie
""", title=temp_movie)

In [14]:
for record in records:
    node = record["movie"]

    print(node.element_id)      # (1)
    print(node.labels)          # (2)
    print(node.items())         # (3)

    # (4)
    print(node["name"])
    print(node.get("name", "N/A"))

4:6636a433-9f85-4f47-b4a7-e2cd79149f79:7594
frozenset({'Movie'})
dict_items([('languages', ['English', ' Japanese', ' French']), ('plotEmbedding', [0.0007911681313998997, -0.029693223536014557, 0.003001880832016468, -0.03651745617389679, -0.006674463860690594, 0.018610358238220215, -0.031750913709402084, -0.02082432620227337, -0.020641999319195747, -0.009663321077823639, 0.021384330466389656, 0.02465319074690342, 0.019743388518691063, -0.013778697699308395, 0.026958322152495384, -0.013016832061111927, 0.023494111374020576, -0.006622370332479477, 0.011740544810891151, -0.006544230505824089, -0.01416939776390791, 0.0003668920253403485, 0.005889807362109423, 0.018753614276647568, 0.013596370816230774, -0.014885681681334972, 0.016005689278244972, -0.01452102791517973, 0.02883368358016014, -0.0027707163244485855, 0.028937870636582375, 0.00478933472186327, -0.002344201784580946, -0.005990738049149513, -0.026476457715034485, -0.0215796809643507, 0.012417757883667946, -0.030995558947324753, 0.

In [15]:
acted_in = record["actedIn"]

print(acted_in.id)         # (1)
print(acted_in.type)       # (2)
print(acted_in.items())    # (3)

# 4
print(acted_in["roles"])
print(acted_in.get("roles", "(Unknown)"))

print(acted_in.start_node) # (5)
print(acted_in.end_node)   # (6)

152255
ACTED_IN
dict_items([('role', 'Ariadne')])
None
(Unknown)
<Node element_id='4:6636a433-9f85-4f47-b4a7-e2cd79149f79:23689' labels=frozenset({'Actor', 'Person'}) properties={'bornIn': 'Halifax, Nova Scotia, Canada', 'tmdbId': '27578', 'imdbId': '0680983', 'born': neo4j.time.Date(1987, 2, 21), 'name': 'Ellen Page', 'bio': "Ellen Philpotts-Page (born February 21, 1987), known professionally as Ellen Page, is a Canadian actress.  Page received both Golden Globe and Academy Award nominations for Best Actress for her role as the title character in the film Juno. \n\nIn 2008, Page was nominated for Time's 100 Most Influential People list and placed #86 on FHM's Sexiest Women in the World list, and moved up to #70 for 2010.  In June 2008, Page was named on Entertainment Weekly's future A-List stars list...", 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/xYMqDWthY4mnbVptrnvOZms9FYt.jpg', 'url': 'https://themoviedb.org/person/27578'}>
<Node element_id='4:6636a433-9f85-4f47-b4a7-

C:\Users\haoch\AppData\Local\Temp\ipykernel_27776\4041262460.py:3: DeprecationWarning: `id` is deprecated, use `element_id` instead
  print(acted_in.id)         # (1)
